In [1]:
import cosmohmc

In [6]:
cosmohmc.sampler.hmcsampler(data="something").sample()

'Sampling from the universe!'

In [2]:
from cosmohmc import hmcsampler

In [4]:
from cosmohmc import HmcSampler

# Assuming you have some cosmological data to analyze
# For demonstration, let's just use a placeholder for the data
cosmological_data = "Placeholder for actual data"

# Initialize the sampler with your data
sampler = HmcSampler(data=cosmological_data)

# Use the sample method
results = sampler.sample()

# Print the results
print(results)

Sampling from the universe!
